In [1]:
"""Global module calls"""

import numpy as np
import sympy as sp
import matplotlib.pyplot as plt 
import inspect
import scipy.integrate

from numpy import exp
from sympy.physics.hydrogen import Psi_nlm
from sympy import symbols, lambdify


In [2]:
"""Classes"""

class FCC:
    """
    Creates an object (whatever the user defines 'self' as) that has tied to it the array of points that are in an FCC
    lattice as well as the vectors needed that create that lattice from a superposition of basis vectors.
    
    ---
    Parameters:
    a, float: The value of the lattice constant along the x axis of the crystal geometry.
    b, float: The value of the lattice constant along the y axis of the crystal geometry.
    c, float: The value of the lattice constant along the z axis of the crystal geometry.
    basis_vectors, list/array: A list/array of the points that can be used to construct the 'lattice_vectors.'
    lattice_points, list/array: A list/array of the points that make up the lattice.
    lattice_vectors, list/array: A list/array of the vectors that can be used to construct the lattice.
    """
    
    def __init__(self, a, b, c):
        """
        Initializes all properties of the FCC crystal lattice object tied to 'self.'
        
        # a, float: The value of the lattice constant along the x axis of the crystal geometry.
        # b, float: The value of the lattice constant along the y axis of the crystal geometry.
        # c, float: The value of the lattice constant along the z axis of the crystal geometry.
        
        ---
        Returns: Nothing since this function initializes the object and the properties of the object.
        
        """
        
        self.a = a
        self.b = b
        self.c = c
        self.cryst_basis = [[a/2, b/2, 0], [a/2, 0, c/2], [0, b/2, c/2]]
        self.lat_pnts = self.fcc()
        self.recip = self.bcc()

        
#     def cryst_vecs(self):
#         """
#         Uses a not-necessarily-orthogonal basis governed by the crystal lattice to recreate the lattice of points in
#         'create_vecs.'
        
#         ---
#         Returns: a/an list/array of the vectors needed to translate to every lattice point from one lattice point. These
#         lattice points can also be used to create the lattice if one starts at one of the bottom corners of the lattice.
#         Realized that this function is totally useless but I will keep it here just in case. It does have outdated
#         notation though.
#         """        
        
#         # Initializes the list/array.
#         lat_vec = []
        
#         # Literal hard coding of the points as vectors. Technically, the vectors I was trying to find are different but since the code cannot tell that, it does not matter.
#         for i in range(len(self.lattice_points)):
#             q, r, s = self.lattice_points[i][0], self.lattice_points[i][1], self.lattice_points[i][2]
#             l, m, n = ((r/self.b) - (s/self.c) + (q/self.a)), ((s/self.c) + (q/self.a) - (r/self.b)), ((r/self.b) + (s/self.c) - (q/self.a))
#             lat_vec.append((l * np.array(self.basis_vectors[0]) + m * np.array(self.basis_vectors[1]) + n * np.array(self.basis_vectors[2])))
            
#         return np.array(lat_vec)
    
    def fcc(self):
        """
        Creates a list of the points on the Face Centered Cubic crystal lattice.
        
        ---
        Parameters:
        self, object: The object itself.
        
        ---
        Returns: A list of vectors that point to each point of the crystal lattice using a standard Cartesian basis.
        """
        
        ####### CONVERT THESE TO SPHERICAL! ########
        
        fcc_lattice_points = [[0, 0, 0], [self.a, 0, 0], [0, self.b, 0], [0, 0, self.c], [self.a, self.b, 0], [self.a, 0, self.c], [0, self.b, self.c], [self.a, self.b, self.c],
                         [self.a/2, self.b/2, 0], [self.a/2, 0, self.c/2], [0 , self.b/2, self.c/2], [self.a/2, self.b/2, self.c], [self.a/2, self.b, self.c/2], [self.a, self.b/2, self.c/2]]
        
        return fcc_lattice_points
    
    def bcc(self):
        """
        Creates a list of the points in the reciprocal lattice of the original FCC lattice. Thankfully, no math has to be
        done because this is simply a BCC lattice. I am most likely incorrect but these could serve as k values for the
        Brillouin Zone.
        
        ---
        Parameters:
        self, object: The object itself.
        
        ---
        Returns: A list of the vectors in the BCC lattice.
        """
        
        ####### CONVERT THESE TO SPHERICAL! ########
        
        bcc_lattice_points = [[0, 0, 0], [self.a, 0, 0], [0, self.b, 0], [0, 0, self.c], [self.a, self.b, 0] ,[self.a, 0, self.c], [0, self.b, self.c], [self.a, self.b, self.c], [self.a/2, self.b/2, self.c/2]]
        
        return bcc_lattice_points

In [20]:
"""Functions"""

def atm_E(n, Z):
    """
    Creates the energy value of the Hydrogenic atomic wavefunctions.
    
    ---
    Parameters:
    n, int: The value of the quantum number n of the orbital.
    Z, int: The number of protons in nucleus of the atom.
    
    ---
    Returns: A scalar which represents the value of the energy of the atomic orbitals.
    """
    ########################################################################################
    # Calling the energy term from the scipy module.
    ########################################################################################
    from scipy.physics.hydrogen import E_nl
    
    return E_nl(n, Z)

def check_list_ok(list_element): 
    # list_element must be [x,y]
    # traverse in the list 
    if abs(list_element[1])<=list_element[0]:
        return True 
    else: return False

def q_num_gen(n):
    """
    Creates all possible combinations of [l,m] for a given quantum n
    
    ---
    Parameters:
    n, int: The value of the quantum number n of the orbital.
    
    ---
    Returns: A list of possible [l,m]
    """
    ########################################################################################
    # arrays filled with the quantum numbers made possible by 'n'.
    # arrange l and m in arrays to access more easily
    ########################################################################################
    l = np.arange(n); m = np.arange(1-n,n+1)
    
    #meshgrid will create arrays of the elements in l and m, which are then reshaped to [x,y] style
    mesh = np.array(np.meshgrid(l, m))
    q_num = mesh.T.reshape(-1, 2); bool_q = np.zeros((len(q_num), 2))
    
    #remove the possibility of abs(m)<l. add zero to remove the negative zeros while preserving all other ints
    for i in range(len(q_num)-1): 
        if check_list_ok(q_num[i])==True: bool_q[i] = True
    q2 = q_num * bool_q + 0
    
    #remove duplicates
    new_q = np.unique(q2,axis=0)
    
    return new_q

def wf_gen(n,a,q_num,Z):
    """
    generates hydrogenic wavefunctions from sympy's Psi_nlm. 
    
    ---
    Parameters:
    n, int: The value of the quantum number n of the orbital.
    a, int: the index of the desired object
    q_num, list: an array of [[l0,m0],[l1,m1],[l2,m2], ...]
    Z, int: atomic number
    ---
    Returns: A function, psi_m(r) for each [n,l,m].  uses sympy.lambdify to turn into a
    numpy-compatible function
    """
    ########################################################################################
    # Creating variables in a way that Psi_nlm can use
    ########################################################################################
    r=symbols("r", real=True, positive=True)
    phi=symbols("phi", real=True)
    theta=symbols("theta", real=True)
    
    lm =q_num[a] 
    wavefunction = Psi_nlm(n, lm[0], lm[1], r, phi, theta, Z)
    lambdified_wf = sp.lambdify((r,phi,theta),wavefunction,modules="numpy")
        
    return wavefunction, lambdified_wf


############################################################################################
# epsiolon(k) = overlap + crystal_field + bond_energy + e_m
############################################################################################

def overlap(k, R_j, n, Z):

    """
    Creates an array of the values of the overlap integral multiplied by the summation term.
    
    ---
    Parameters:
    k, list: A list of the Brillouin Zone (k-space) values of interest.
    R_j, list: A  list of the lattice points in the real space crystal lattice.
    n, int: The value of the quantum number n of the orbital.
    Z, int: The number of protons in nucleus of the atom.
    
    ---
    Returns: A list of the values for the overlap term to be plotted in k-space.
    """
    x=[]
    q_num = q_num_gen(n)
    
    ########################################################################################    
    # for our n=3 case, there are 9 terms.  i will generate them "by hand" for now.
    # w_n is the symbolic function; f_n is the numpy one
    # its going to be a pain in ass to integrate these.  Collin is probably right about Psi_nlm
    # we may be able to use sympy's R_nl, and do the radial and angular parts separately
    #    to avoid some of these problems
    ########################################################################################
    
    w1, f1 = wf_gen(n,0,q_num,Z)   
    w2, f2 = wf_gen(n,1,q_num,Z)
    w3, f3 = wf_gen(n,2,q_num,Z)  
    w4, f4 = wf_gen(n,3,q_num,Z)  
    w5, f5 = wf_gen(n,4,q_num,Z)  
    w6, f6 = wf_gen(n,5,q_num,Z)  
    w7, f7 = wf_gen(n,6,q_num,Z)  
    w8, f8 = wf_gen(n,7,q_num,Z)  
    w9, f9 = wf_gen(n,8,q_num,Z)  
       
    return w4, f4
    

    
    

def crystal_field(R_j):
    
    """
    Stuff.
    
    ---
    Returns: An array of 
    """
    
    pass

def bond_energy():
    
    """
    Stuff.
    """
    
    pass
    
def plotting(n):
    
    """
    Stuff.
    
    ---
    Return: Plots of energy arrays.
    """
    
    "Stuff."
            
    "plot"

In [19]:
#testing

fcc = FCC(0.352, 0.352, 0.352)
R_j = fcc.lat_pnts
k_j =fcc.recip

w, f = overlap(k_j, R_j, 3, 28)
###################################################################################
# Verifying that the "numpy'd" function is the same as the symbolic wavefunction
###################################################################################

r=symbols("r", real=True, positive=True)
phi=symbols("phi", real=True)
theta=symbols("theta", real=True)
    
i = sp.integrate(w, (r,0,np.inf))
print(i)
j = f(1,0,0)
print(j)

0


NameError: name 'assoc_legendre' is not defined

In [16]:
print(w)

76.8247788101718*r**1.0*(4.0 - 18.6666666666667*r)*exp(-28*r/3)*exp(1.0*I*phi)*assoc_legendre(1.0, 1.0, cos(theta))/sqrt(pi)


In [17]:
inspect.getsource(f)


'def _lambdifygenerated(r, phi, theta):\n    return ((1/2)*(3311.82556891097*r**2 - 1064.51536143567*r + 57.0276086483393)*exp(-28/3*r)/sqrt(pi))\n'